# Retrieving Participant Warnings for Mercury Challenge Using Python

This notebook will walk through the process of retrieving a particpant's Mercury Challenge warnings using the Python's **Requests** library.  As with warning submissions a challenge participant must be registered with TopCoder and be in possession of an API key, which will be used to authenticate submissions.  For more information on this please see https://iarpa.gov/challenges/mercury.html and TopCoder's instructions at https://docs.google.com/document/d/1KViMdqLFecnjEur6pE6-fHKcoO94PkK1g7XWwpEyCOE/edit

In this notebook we will use the fictional participant name "testuser" who has obtained the API key "mysecretkey" after registering for the challenge.  This participant has previously submitted warnings and wants to examine the warning submission history.

The history retrieval URL for the challenge is https://87g554i96c.execute-api.us-east-1.amazonaws.com/Production/warning/history.  Submissions must use **GET** and must have the following header:
- "x-api-key:TopCoderusername:token"


Note that you will not be able to run these commands directly from this notebook without modification.  You will need to use your own user name and token and you'll need to modify the query content.

### Required Elements

- `participant_id`
- `start_date`
- `end_date`
- `page` (This uses zero-based indexing, i.e. the first page is numbered `0`)

### Optional Elements

- `event_type`: If present, must be one of `Military Activity`, `Civil Unrest`, or `Disease`
- `country`
- `state`: For `event_type=CU` this must be one of `Amman`, `Irbid`, `Madaba`
- `city`:  For `event_type=CU` this must be `Tahrir Square`

## Example Using curl

## Using Python

In [1]:
import requests
from uuid import uuid4 #Used to generate unique warning IDs
from pprint import pprint
from collections import OrderedDict
import re

secret_token = "mysecretkey"
userid = "testuser"
HISTORY_URL = "https://87g554i96c.execute-api.us-east-1.amazonaws.com/Production/warning/history"
headers = {'x-api-key':'{0}:{1}'.format(userid, secret_token)}

Below we build a Python **dict** for our query submission. The keys for this dict will include any required and optional query elements.  In this example we use an OrderedDict to preserve the order of the dict keys.

In [2]:
query_dict = OrderedDict()

query_dict["participant_id"] = userid
query_dict["start_date"] = "2010-01-01"
query_dict["end_date"] = "2020-08-31"
query_dict["event_type"] = "Military Activity"
query_dict["country"] = "Syria"
query_dict["page"] = 0
pprint(query_dict)

OrderedDict([('participant_id', 'testuser'),
             ('start_date', '2010-01-01'),
             ('end_date', '2020-08-31'),
             ('event_type', 'Military Activity'),
             ('country', 'Syria'),
             ('page', 0)])


We now convert the `query_dict` to a string and append it to the history URL.

In [3]:
query_str = "&".join(["=".join((k, str(v))) for k,v in query_dict.items()])
print(query_str)

url = HISTORY_URL + "?" + query_str
url = re.sub(" ", "%20", url)
print("Revised URL is {}".format(url))

participant_id=testuser&start_date=2010-01-01&end_date=2020-08-31&event_type=Military Activity&country=Syria&page=0
Revised URL is https://87g554i96c.execute-api.us-east-1.amazonaws.com/Production/warning/history?participant_id=testuser&start_date=2010-01-01&end_date=2020-08-31&event_type=Military%20Activity&country=Syria&page=0


Using the `get` method for requests we make our query.  We can then examine the `json` attribute of the returned results to see if our submission was successful.

In [4]:
result = requests.get(url, headers=headers)

pprint(result.json())

['Page: 0 of 0',
 {'Actor': 'Syrian Arab Military',
  'City': 'Al Maḩmūdīyah',
  'Country': 'Syria',
  'Event_Date': '2016-02-18',
  'Event_Subtype': 'Conflict',
  'Event_Type': 'Military Activity',
  'Latitude': 35.9143,
  'Longitude': 38.0711,
  'State': 'Aleppo',
  'Warning_ID': 'BaserateConflict1',
  'latest': True,
  'participant_id': 'testuser',
  'sequence': 1,
  'timestamp': '2018-07-31T18:23:06.253844'},
 {'Actor': 'Syrian Arab Military',
  'City': 'Al Maḩmūdīyah',
  'Country': 'Syria',
  'Event_Date': '2016-02-18',
  'Event_Subtype': 'Conflict',
  'Event_Type': 'Military Activity',
  'Latitude': 35.9143,
  'Longitude': 38.0711,
  'State': 'Aleppo',
  'Warning_ID': 'testhandle_Warning1',
  'latest': True,
  'participant_id': 'testuser',
  'sequence': 1,
  'timestamp': '2018-08-08T16:24:51.788294'},
 {'Actor': 'Syrian Arab Military',
  'City': 'Al Maḩmūdīyah',
  'Country': 'Syria',
  'Event_Date': '2016-02-18',
  'Event_Subtype': 'Conflict',
  'Event_Type': 'Military Activity',

## Results

If your retrieval request is successful you will see an output similar to what is shown below (truncated for clarity).  The first item in the list is a string that tells you which page you are on and how many pages match your query.  The remaining elements are a list of your warnings.

### Problems

Here we describe several errors in output and how to try to address them.

This means that the `participant_id` in your query content doesn't match the participant id in your header.

In [5]:
bad_token = "invalid_token"
bad_headers = {'x-api-key':'{0}:{1}'.format(userid, bad_token),
           'Content-Type': 'application/json'}
bad_result = requests.get(url, headers=bad_headers)
pprint(bad_result.json())

{'message': 'Forbidden'}


This indicates a mismatch between the participant ID and the API Key used in the request header.

In [6]:
query_dict = OrderedDict()

query_dict["participant_id"] = userid
query_dict["start_date"] = "2010-01-01"
query_dict["end_date"] = "2020-08-31"
query_dict["event_type"] = "Bogus Event Type"
query_dict["country"] = "Syria"
query_dict["page"] = 0
query_str = "&".join(["=".join((k, str(v))) for k,v in query_dict.items()])
url = HISTORY_URL + "?" + query_str
url = re.sub(" ", "%20", url)
bad_result = requests.get(url, headers=headers)
pprint(bad_result.json())

[]


This is a valid result, it just means that your query didn't return any results at all.  Check for legal event type, country, date range, and page.